In [2]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
from datetime import datetime
import time

# 행 많이보기
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

# 한글 폰트 설정

fm.get_fontconfig_fonts()
font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
plt.rc('font', family=font_name)
plt.rc('font', size=12)
plt.figure(figsize=(20,10))

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

In [3]:
# 필요한 모듈 불러오기
import requests
from bs4 import BeautifulSoup
import pandas as pd

http://www.nsmall.com/NSSubBrandPageLayoutView?whereGo=go2&brandCode=643677&searchType=search&langId=-9&catalogId=97001&storeId=13001&cate1Code=1583570#


http://www.nsmall.com/NSSubBrandPageLayoutView?whereGo=go2&brandCode=829310&searchType=search&langId=-9&catalogId=97001&storeId=13001&cate1Code=1583567#;


http://www.nsmall.com/NSSubBrandPageLayoutView?whereGo=go2&brandCode=833224&searchType=search&langId=-9&catalogId=97001&storeId=13001&cate1Code=1583566

http://www.nsmall.com/NSSubBrandPageLayoutView?whereGo=go2&brandCode=818598&searchType=search&langId=-9&catalogId=97001&storeId=13001&cate1Code=1583578

* LG 생활건강 : 100330 / 452334
* LG 퓨리케어 : 100153 / 802322


http://www.nsmall.com/NSSubBrandPageLayoutView?whereGo=go2&

brandCode=452334& # 브랜드코드

searchType=search&langId=-9

&catalogId=97001

&storeId=13001

&cate1Code=1583570 # 뷰티 카테고리



http://www.nsmall.com/ProductDisplay?

catalogId=97001&busChnId=INT&langId=-9&storeId=13001&

partNumber=29879654& # 상품코드

menuUri=NSItemDetailView

&cate1Code=1583570 # 카테고리


http://www.nsmall.com/ProductDisplay?

catalogId=97001&busChnId=INT&langId=-9&storeId=13001&

partNumber=29879653&

menuUri=NSItemDetailView&

cate1Code=1583570



http://www.nsmall.com/NSSubBrandPageLayoutView?

whereGo=go2&

brandCode=452339

&searchType=search&langId=-9&catalogId=97001&storeId=13001&

cate1Code=1583565

http://www.nsmall.com/NSSubBrandPageLayoutView?

whereGo=go2&

brandCode=802322

&searchType=search&langId=-9&catalogId=97001&storeId=13001&

cate1Code=1583565

In [ ]:
# url 시작
url1 = 'http://www.nsmall.com/NSSubBrandPageLayoutView?whereGo=go2&brandCode=643677&searchType=search&langId=-9&catalogId=97001&storeId=13001&cate1Code=1583570#'

In [ ]:

url3 = '&boardCd=&isBoard=false'
df = []
all_url = []

for url2 in range(50):
    url = url1 + str(url2) + url3
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
    html = list(soup.children)[3]
    body = list(html.children)[3]
    body2 = list(body.children)[7]
    body3 = list(body2.children)[3]
    body4 = list(body3.children)[7]
    turn = len(list(body4.select('span.list_subject > a')))

    for tag in range(0,turn):
        # 제목
        a = body4.select('span.list_subject > a')[tag].text
        
        # 링크
        b = body4.select('span.list_subject > a')[tag].get('href')
       
        # 댓글 수
        if len(list(body4.select('div.list_title.oneline')[tag])) == 9: # 그림 ㅇ , 댓글 ㅇ
            c = list(body4.select('div.list_title.oneline')[tag])[5].text.strip('\n')
        elif len(list(body4.select('div.list_title.oneline')[tag])) == 7: 
            area = list(body4.select('div.list_title.oneline')[tag]) 
            if str(area[3])[13] is "i": # 그림 ㅇ, 댓글 x
                c = '0'
            else:   # 그림 x, 댓글 ㅇ
                c = list(body4.select('div.list_title.oneline')[tag])[3].text.strip('\n')
        else:                                                       
            c = '0' # 그림 x, 댓글 x
       
        # 글쓴이
        if list(body4.select('span.nickname')[tag])[1].text:
            d = list(body4.select('span.nickname')[tag])[1].text
        else:
            d = list(body4.select('span.nickname')[tag])[1].get('alt')
       
        # 조회수
        e = body4.select('span.hit')[tag].text
        if "k" in e:
            e = str(int(float(e[:4]) * 1000))
        else:
            pass
       
        # 글 작성 시간
        f = body4.select('span.timestamp')[tag].text

        df.append([a,b,c,d,e,f])
        all_url.append(b)

print(len(df))

In [ ]:
%%time

num = len(all_url)
all_content = []
for url_num in range(920):
    print(url_num)
    each_url = "https://www.clien.net/" + all_url[url_num]
    req = requests.get(each_url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')

    html = list(soup.children)[3]
    body = list(html.children)[3]
    body2 = list(body.children)[7]
    body3 = list(body2.children)[3]
    body4 = list(body3.children)[1]

    

    # 글 내용
    content = []
    tmp = list(body4.find_all('p'))
    for i in range(len(tmp)):
        if tmp[i].text is "" or str(tmp[i].text)[3:] == 'GIF' or str(tmp[i].text)[:4] == 'http':
            pass
        else:
            tmpstr = str(tmp[i].text)
            tmpstr = tmpstr.replace('\n\n','')
            tmpstr = tmpstr.replace('\xab','')
            tmpsave = tmpstr.replace('\xa0','')
            content.append(tmpsave)
    df[url_num].append(content)

    # 공감수
    sympathy = body4.select('#comment-head > a > strong')[0].text
    df[url_num].append(sympathy)
    # 댓글 내용
    reply_cnt = int(df[url_num][2])
    reply_content = []
    if reply_cnt != 0:
        replies = body4.select('#div_content div div div div.comment_view')
        for reply in replies:
            reply_content.append(reply.get_text(strip=True).replace('\n', '').replace('\r', '').replace('\t', ''))
    reply_content = '\n'.join(reply_content)
    df[url_num].append(reply_content)

    print(url_num, "finished")